<a href="https://colab.research.google.com/github/KirstenSchulz/AIF360/blob/master/examples/Identifying_Bias_and_Correcting_for_Fairness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project 1 - Identifying Bias and Correcting for Fairness
 by Kirsten Schulz

##Part 1 - Overview of AIF360


This work is based on a paper by IBM, AI Fairness 360: An Extensible Toolkik for Detecting, Understanding, and Mitigating Unwanted Algororitmic Bias.
<p>
Fairness is a complex and multi-faced concept; therefore there is not one definition, neither one measure, and different definitions produce different outcomes. The burden is on Machine Learning (ML) and Artificial Intelligence (AI) developers. They have to decide if they should debiase the data, the model or the predictions.
<p>
Some of the terminology that we are using in this notebook are listed below.
<p>
<b>Favorable label</b> is a label whose value corresponds to an outcome that provides an advantage to the recipient.
<p>
<b>Protected attribute</b> is an attribute that partitions a population into groups that have parity in terms of the benefit received.
<p>
<b>Priviledge value</b> of a protected attribute is a group that has historically been at systematic advantage.
<p>
<b>Group fairness</b> is the goal of the groups defined by protected attributes receiving similar treatment or outcomes. 
<p>
<b>Individual fairness</b> is the goal of similar individuals receiving similar treatment or outcomes.
<p>
<b>Bias</b> is a sytematic error. 
<p>
<b>Fairness metrics</b> is a quantification of unwanted bias in training data or models.
<p>
<b>Bias mitigation algorithms </b> are procedures for reducing unwanted bias in training data or models.
<p> There are three places to work towards achieving fairnes with AIF360: during pre-processing, in-processing, and post-processinng. In other words we can apply AIF360 mitigation algorithms to the training data, to the learning model, and to the predictions. Where to do so will depend in what is available to the developer and/or user. All the processes  can be viewed with same relevance by AIF360 but their relevance will also depend on the domain knowledge of the user.
<p>
The dataset used in this notebook will be structured and it is the German Credit data set.
<p>
The Metric class and subclasses implement 71 bias metrics. They are often related to the confusion matrix.
<p>
This assignment has given us awareness of the existence of bias in ML and insights into its mitigation.

In [0]:
#!pip uninstall aif360[all] -y
#!ls /usr/local/lib/python3.6/dist-packages/aif*

In [0]:
!pip install 'aif360[all]'

##Part 2 - Executing the credit decision pipeline
The credit decision pipeline detects bias and removes it using the reweighting algorithm. This algorithm generates weights for the examples per group before training. The following code is practically a copy of the code provided to us from the example notebook "Detecting and mitigating age bias on credit decisions." I only added a few lines to be able to execute it.

In [0]:
# Load all necessary packages
import sys
sys.path.insert(1, "../")  

import numpy as np
np.random.seed(0)

from aif360.datasets import GermanDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import  load_preproc_data_german

from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt

import tensorflow as tf


In [0]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.doc
!sudo cp german.data /usr/local/lib/python3.6/dist-packages/aif360/data/raw/german

--2020-06-14 21:21:25--  https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79793 (78K) [application/x-httpd-php]
Saving to: ‘german.data.4’

german.data.4       100%[===================>]  77.92K   276KB/s    in 0.3s    

2020-06-14 21:21:26 (276 KB/s) - ‘german.data.4’ saved [79793/79793]

--2020-06-14 21:21:27--  https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.doc
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4679 (4.6K) [application/x-httpd-php]
Saving to: ‘german.doc.4’

german.doc.4        100%[===================>]   4.57K  --.-KB/s    i

#### Load dataset and set options

In [0]:
dataset_orig = GermanDataset(
    protected_attribute_names=['age'],           # this dataset also contains protected
                                                 # attribute for "sex" which we do not
                                                 # consider in this evaluation
    privileged_classes=[lambda x: x >= 25],      # age >=25 is considered privileged
    features_to_drop=['personal_status', 'sex']  # ignore sex-related attributes

)

dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

In [0]:
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())


#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.169905


In [0]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
dataset_transf_train = RW.fit_transform(dataset_orig_train)

In [0]:
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                               unprivileged_groups=unprivileged_groups,
                                               privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())

#### Transformed training dataset

Difference in mean outcomes between unprivileged and privileged groups = 0.000000


####Findings
<p>Before applying reweighting, the difference between the mean outcomes is -0.169905 (in favor of the people 25 years or older). After applying the reweighting algorithm the favorable outcome rates are equal (the difference is zero). The bias in the training data for that particular attribute has been mitigated.

##Part 3 - Applying adversarial debiasing
I have chosen adversarial debiasing to explore the in-process method but soon I encountered that the German data set is small  and in some occasions there was not much mitigation of bias and the software even  had a division by zero. I have decided to stick to it in spite of the size issue because of the learning experience. I played with the definition of the priviledge class (anyway this class definition varies from culture to culture) to compensate for it, and with the split of the data set. 

#### Load dataset and set options


In [0]:
dataset_orig = GermanDataset(
    protected_attribute_names=['age'],           # this dataset also contains protected
                                                 # attribute for "sex" which we do not
                                                 # consider in this evaluation
    privileged_classes=[lambda x: x >= 30],      # age >=25 is considered privileged
    features_to_drop=['personal_status', 'sex'] # ignore sex-related attributes

)

dataset_orig_train, dataset_orig_test = dataset_orig.split([0.60], shuffle=True)

privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

In [0]:
# print out some labels, names, etc.
display(Markdown("#### Training Dataset shape"))
print(dataset_orig_train.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig_train.favorable_label, dataset_orig_train.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_orig_train.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_orig_train.privileged_protected_attributes, 
      dataset_orig_train.unprivileged_protected_attributes)
display(Markdown("#### Dataset feature names"))
print(dataset_orig_train.feature_names)

#### Training Dataset shape

(600, 57)


#### Favorable and unfavorable labels

1.0 2.0


#### Protected attribute names

['age']


#### Privileged and unprivileged protected attribute values

[array([1.])] [array([0.])]


#### Dataset feature names

['month', 'credit_amount', 'investment_as_income_percentage', 'residence_since', 'age', 'number_of_credits', 'people_liable_for', 'status=A11', 'status=A12', 'status=A13', 'status=A14', 'credit_history=A30', 'credit_history=A31', 'credit_history=A32', 'credit_history=A33', 'credit_history=A34', 'purpose=A40', 'purpose=A41', 'purpose=A410', 'purpose=A42', 'purpose=A43', 'purpose=A44', 'purpose=A45', 'purpose=A46', 'purpose=A48', 'purpose=A49', 'savings=A61', 'savings=A62', 'savings=A63', 'savings=A64', 'savings=A65', 'employment=A71', 'employment=A72', 'employment=A73', 'employment=A74', 'employment=A75', 'other_debtors=A101', 'other_debtors=A102', 'other_debtors=A103', 'property=A121', 'property=A122', 'property=A123', 'property=A124', 'installment_plans=A141', 'installment_plans=A142', 'installment_plans=A143', 'housing=A151', 'housing=A152', 'housing=A153', 'skill_level=A171', 'skill_level=A172', 'skill_level=A173', 'skill_level=A174', 'telephone=A191', 'telephone=A192', 'foreign_wor

#### Metric for original test data 

In [0]:
# Metric for the original dataset
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())
metric_orig_test = BinaryLabelDatasetMetric(dataset_orig_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_test.mean_difference())

#### Original training dataset

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.078457
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.159340


In [0]:
min_max_scaler = MaxAbsScaler()
dataset_orig_train.features = min_max_scaler.fit_transform(dataset_orig_train.features)
dataset_orig_test.features = min_max_scaler.transform(dataset_orig_test.features)
metric_scaled_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
display(Markdown("#### Scaled dataset - Verify that the scaling does not affect the group label statistics"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_scaled_train.mean_difference())
metric_scaled_test = BinaryLabelDatasetMetric(dataset_orig_test, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_scaled_test.mean_difference())


#### Scaled dataset - Verify that the scaling does not affect the group label statistics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.078457
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.159340


### Learn plain classifier without debiasing

In [0]:
# Load post-processing algorithm that equalizes the odds
# Learn parameters with debias set to False
sess = tf.Session()
plain_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='plain_classifier',
                          debias=False,
                          sess=sess)

In [0]:
plain_model.fit(dataset_orig_train)




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





epoch 0; iter: 0; batch classifier loss: 0.671214
epoch 1; iter: 0; batch classifier loss: 0.536403
epoch 2; iter: 0; batch classifier loss: 0.537950
epoch 3; iter: 0; batch classifier loss: 0.516291
epoch 4; iter: 0; batch classifier loss: 0.507715
epoch 5; iter: 0; batch classifier loss: 0.468882
epoch 6; iter: 0; batch classifier loss: 0.522880
epoch 7; iter: 0; batch classifier loss: 0.524104
epoch 8; iter: 0; b

In [0]:
# Apply the plain model to test data
dataset_nodebiasing_train = plain_model.predict(dataset_orig_train)
dataset_nodebiasing_test = plain_model.predict(dataset_orig_test)

In [0]:
# Metrics for the dataset from plain model (without debiasing)
display(Markdown("#### Plain model - without debiasing - dataset metrics"))
metric_dataset_nodebiasing_train = BinaryLabelDatasetMetric(dataset_nodebiasing_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Train set: Difference in mean predicted outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_train.mean_difference())

metric_dataset_nodebiasing_test = BinaryLabelDatasetMetric(dataset_nodebiasing_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean predicted outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_test.mean_difference())

display(Markdown("#### Plain model - without debiasing - classification metrics"))
classified_metric_nodebiasing_test = ClassificationMetric(dataset_orig_test, 
                                                 dataset_nodebiasing_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_nodebiasing_test.accuracy())
TPR = classified_metric_nodebiasing_test.true_positive_rate()
TNR = classified_metric_nodebiasing_test.true_negative_rate()
bal_acc_nodebiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_nodebiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_nodebiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_nodebiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_nodebiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_nodebiasing_test.theil_index())

#### Plain model - without debiasing - dataset metrics

Train set: Difference in mean predicted outcomes between unprivileged and privileged groups = -0.067344
Test set: Difference in mean predicted outcomes between unprivileged and privileged groups = -0.147563


#### Plain model - without debiasing - classification metrics

Test set: Classification accuracy = 0.725000
Test set: Balanced classification accuracy = 0.663337
Test set: Disparate impact = 0.818770
Test set: Equal opportunity difference = -0.055758
Test set: Average odds difference = -0.109869
Test set: Theil_index = 0.133592


### Apply in-processing algorithm based on adversarial learning

In [0]:
sess.close()
tf.reset_default_graph()
sess = tf.Session()

In [0]:
# Learn parameters with debias set to True
debiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='debiased_classifier',
                          debias=True,
                          #batch_size=50,
                          #classifier_num_hidden_units=200,
                          sess=sess)

In [0]:
debiased_model.fit(dataset_orig_train)

epoch 0; iter: 0; batch classifier loss: 0.867398; batch adversarial loss: 0.684761
epoch 1; iter: 0; batch classifier loss: 0.695902; batch adversarial loss: 0.685513
epoch 2; iter: 0; batch classifier loss: 0.596855; batch adversarial loss: 0.681464
epoch 3; iter: 0; batch classifier loss: 0.564710; batch adversarial loss: 0.677372
epoch 4; iter: 0; batch classifier loss: 0.528753; batch adversarial loss: 0.682609
epoch 5; iter: 0; batch classifier loss: 0.526863; batch adversarial loss: 0.678734
epoch 6; iter: 0; batch classifier loss: 0.517595; batch adversarial loss: 0.685336
epoch 7; iter: 0; batch classifier loss: 0.536575; batch adversarial loss: 0.683176
epoch 8; iter: 0; batch classifier loss: 0.592748; batch adversarial loss: 0.683826
epoch 9; iter: 0; batch classifier loss: 0.481071; batch adversarial loss: 0.651167
epoch 10; iter: 0; batch classifier loss: 0.498162; batch adversarial loss: 0.666030
epoch 11; iter: 0; batch classifier loss: 0.524066; batch adversarial loss:

In [0]:
# Apply the plain model to test data
dataset_debiasing_train = debiased_model.predict(dataset_orig_train)
dataset_debiasing_test = debiased_model.predict(dataset_orig_test)

In [0]:
# Metrics for the dataset from plain model (without debiasing)
display(Markdown("#### Plain model - without debiasing - dataset metrics"))
print("Train set: Difference in mean predicted outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_train.mean_difference())
print("Test set: Difference in mean predicted outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_test.mean_difference())

# Metrics for the dataset from model with debiasing
display(Markdown("#### Model - with debiasing - dataset metrics"))
metric_dataset_debiasing_train = BinaryLabelDatasetMetric(dataset_debiasing_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Train set: Difference in mean predicted outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_train.mean_difference())

metric_dataset_debiasing_test = BinaryLabelDatasetMetric(dataset_debiasing_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean predicted outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_test.mean_difference())



display(Markdown("#### Plain model - without debiasing - classification metrics"))
print("Test set: Classification accuracy = %f" % classified_metric_nodebiasing_test.accuracy())
TPR = classified_metric_nodebiasing_test.true_positive_rate()
TNR = classified_metric_nodebiasing_test.true_negative_rate()
bal_acc_nodebiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_nodebiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_nodebiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_nodebiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_nodebiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_nodebiasing_test.theil_index())



display(Markdown("#### Model - with debiasing - classification metrics"))
classified_metric_debiasing_test = ClassificationMetric(dataset_orig_test, 
                                                 dataset_debiasing_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_debiasing_test.accuracy())
TPR = classified_metric_debiasing_test.true_positive_rate()
TNR = classified_metric_debiasing_test.true_negative_rate()
bal_acc_debiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_debiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_debiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_debiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_debiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_debiasing_test.theil_index())

#### Plain model - without debiasing - dataset metrics

Train set: Difference in mean predicted outcomes between unprivileged and privileged groups = -0.067344
Test set: Difference in mean predicted outcomes between unprivileged and privileged groups = -0.147563


#### Model - with debiasing - dataset metrics

Train set: Difference in mean predicted outcomes between unprivileged and privileged groups = -0.044168
Test set: Difference in mean predicted outcomes between unprivileged and privileged groups = -0.120997


#### Plain model - without debiasing - classification metrics

Test set: Classification accuracy = 0.725000
Test set: Balanced classification accuracy = 0.663337
Test set: Disparate impact = 0.818770
Test set: Equal opportunity difference = -0.055758
Test set: Average odds difference = -0.109869
Test set: Theil_index = 0.133592


#### Model - with debiasing - classification metrics

Test set: Classification accuracy = 0.735000
Test set: Balanced classification accuracy = 0.677421
Test set: Disparate impact = 0.847700
Test set: Equal opportunity difference = -0.037640
Test set: Average odds difference = -0.074144
Test set: Theil_index = 0.132249


##Part 4 - Difference between adversarial debiasing and reweighting.
Reweighting detects bias in the training data and mitigates it before the data goes into the model. In part 2 we have seen how it mitigated effectively from -0.169905 to 0.0 the difference between the mean of the outcomes of the unprivilege group and the one of the privilege group. Adversarial debiasing is an in-processing technique that reduces an adversary's ability to determine the protected attribute from the predictions. In this approach the predictions do not carry information that the adversary can exploit, so it mitigates bias during the training. It is a very transparent method but also could improve accuracy. It can be seen in part 3, where the accuracy of the classification  went from 0.725000 to 0.735000. The disparate impact also improved, the closer to 1, the better. It went from 0.818770 to 0.847700. 
By experimenting with the definition of the privilege class, the difference of the mean of the outcomes became very low, so we did not need to apply mitigation on the bias in the training data that much. Instead we could inspect the model itself and we found that we can mitigate bias during the training process and we even could get an improvement in accuracy (from 0.725000 to 0.735000). 
My last thought on this is that given the issue of the size of the German data set, we could split the data so that we could use every sample for training like for example, cross-validation and find if there is need of more mitigation.  


    References:
    [1] B. H. Zhang, B. Lemoine, and M. Mitchell, "Mitigating UnwantedBiases with Adversarial Learning," 
    AAAI/ACM Conference on Artificial Intelligence, Ethics, and Society, 2018.